
Vehicles Exploratory Data Analysis
Project Description

Using a car advertisement dataset (vehicles_us.csv), this TripleTen project aims to provide additional practice on common software engineering tasks to develop and deploy a web application to a cloud service.

For this project, I've created a Streamlit application hosted on Render that allows users in interact with the car advertisement dataset to uncover insights about what car attributes can increase its value on the secondary market. For additional context, the car advertisement dataset contains information on used car sales such as price, model year, model, car condition, number of cylinders, fuel type, miles on the odometer, transmission type, car type, paint color, four wheel drive, date posted, and days listed. Using data visualizations, this project aims to uncover insights about which attributes have a directional impact on price and allows for users to interact with the plots to uncover their own findings.

This notebook contains my exploratory data analysis of the car advertisement dataset. After reading in the vehicles_us.csv and converting to a dataframe, I will check data types, and I will check for and replace any missing values in the dataframe. This EDA will be my first step in understanding, cleaning, and transforming the dataset so that I can create a streamlit web application that contains interactive visualizations.

Import Libraries for Analysis

In [10]:
# Import packages
import pandas as pd 
import streamlit as st
import plotly.express as px
import altair as alt

Read in File

In [11]:
# Read in CSV from local desktop 
vehicles = pd.read_csv('/Users/brandon/tripleten_project_four/TripleTen-Project-4/vehicles_us.csv')

Data Quality Check

In [12]:
# Check data types
vehicles.info()

# Check data description
vehicles.describe(include='all')

# Check sample of data
vehicles.head()

# Check for missing values
vehicles.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

There are missing values in the following fields - model_year, cylinders, odometer, paint_color, is_4wd. In the next section, I will replace these missing values with the following values

    model_year - I will replace null values with a 0. Since we know that there weren't cars made in year 0, this will be a catch-all for cars without a model year
    cylinders - I will replace cylinders with the median value of the same vehcile type that was sold. Making the assumption that the median cylinder by type most closely resemble how many cylinders the vehcile may have, if it had a missing value.
    odometer - I will replace Odometer with the median value of the same vehcile type that was sold. Because Odometer must be an integer and because 0 represents a new car, I will set the missing value with an average value for all sold vehicles in the dataset of the same vehicle type.
    paint_color - I will replace null values with the string 'Unknown' and convert paint_color to be a string.
    is_4wd - I will replace null values with 0. Currently, when is_4wd is 1, this means that the vehicle has four-wheel drive. If zero, we can assume that the vehicle either doesn't have four wheel drive or that the value is unknown.

Fill in Missing Values

In [13]:
# Model Year - replace nulls with 0
vehicles['model_year'] = vehicles['model_year'].fillna(0)

# Cylinders - replace nulls with median by Type
vehicles['type'] = vehicles['type'].astype(str)  # ensure type column is string
vehicles['cylinders'] = vehicles['cylinders'].fillna(vehicles.groupby('type')['cylinders'].transform('median'))

# Odometer - replace nulls with median by Type
vehicles['odometer'] = vehicles['odometer'].fillna(vehicles.groupby('type')['odometer'].transform('median'))

# Paint Color - replace nulls with 'Unknown'
vehicles['paint_color'] = vehicles['paint_color'].fillna('Unknown')

# Is 4WD - replace nulls with 0
vehicles['is_4wd'] = vehicles['is_4wd'].fillna(0)

# Check again for missing values
vehicles.isna().sum()

price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

Convert Data Types

In [14]:
# Convert data types 
vehicles['price'] = vehicles['price'].astype(float)
vehicles['model_year'] = vehicles['model_year'].astype('Int64')
vehicles['model'] = vehicles['model'].astype(str)
vehicles['condition'] = vehicles['condition'].astype(str)
vehicles['cylinders'] = vehicles['cylinders'].astype('Int64')
vehicles['fuel'] = vehicles['fuel'].astype(str)
vehicles['odometer'] = vehicles['odometer'].astype(float)
vehicles['transmission'] = vehicles['transmission'].astype(str)
vehicles['type'] = vehicles['type'].astype(str)
vehicles['paint_color'] = vehicles['paint_color'].astype(str)
vehicles['is_4wd'] = vehicles['is_4wd'].astype('Int64')
vehicles['date_posted'] = pd.to_datetime(vehicles['date_posted'])
vehicles['days_listed'] = vehicles['days_listed'].astype('Int64')

# Rename column names
vehicles = vehicles.rename(columns={"price": "Price", "model_year": "Model Year", "model": "Model", "condition": "Condition", "cylinders": "Cylinders", "fuel": "Fuel", "odometer": "Odometer", "transmission": "Transmission", "type": "Type", "paint_color": "Paint Color", "is_4wd": "Is 4WD", "date_posted": "Date Posted", "days_listed": "Days Listed"})

# Check change to data types 
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Price         51525 non-null  float64       
 1   Model Year    51525 non-null  Int64         
 2   Model         51525 non-null  object        
 3   Condition     51525 non-null  object        
 4   Cylinders     51525 non-null  Int64         
 5   Fuel          51525 non-null  object        
 6   Odometer      51525 non-null  float64       
 7   Transmission  51525 non-null  object        
 8   Type          51525 non-null  object        
 9   Paint Color   51525 non-null  object        
 10  Is 4WD        51525 non-null  Int64         
 11  Date Posted   51525 non-null  datetime64[ns]
 12  Days Listed   51525 non-null  Int64         
dtypes: Int64(4), datetime64[ns](1), float64(2), object(6)
memory usage: 5.3+ MB


Create Visualizations


Create Scatter Plot

Testing creation of scatter plots and histogram for usage in streamlit application. Because I set missing values in Model Year to 0, I will need to create a slider in the streamlit app that allows users to filter out Model Year 0.

In [15]:
# Ensure categorical columns are strings
vehicles['Condition'] = vehicles['Condition'].astype(str)
vehicles['Type'] = vehicles['Type'].astype(str)

# Scatter Plot
fig = px.scatter(
    vehicles,
    x="Cylinders",
    y="Price",
    color="Condition",       # Different colors for each condition
    symbol="Condition",      # Different shapes for each condition
    hover_data=['Model'],
    color_discrete_sequence=px.colors.qualitative.Plotly  # ensures distinct colors
)

fig.show()

Create Histogram Plot

In [16]:
# Ensure Type column is string
vehicles['Type'] = vehicles['Type'].astype(str)

# Histogram
fig = px.histogram(
    vehicles,
    x="Model Year",
    color="Type",
    color_discrete_sequence=px.colors.qualitative.Plotly
)

fig.show()